In [1]:
import sys
import os
sys.path.append('E:\\software_git_repos\\cofe-python-analysis-tools\\utils_meinhold')
sys.path.append('../../cofe-python-analysis-tools/utils_meinhold')
sys.path.append('E:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca')
sys.path.append('../../cofe-python-analysis-tools/utils_zonca')
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca\\pointing')
sys.path.append('../../cofe-python-analysis-tools/utils_zonca/pointing')
#sys.path.append('D:\\software_git_repos\\greenpol')
sys.path.append('../')
#sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\')
sys.path.append('../telescope_control')
#sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\VtoT\\')
sys.path.append('../VtoT')
import convert
import cofe_util as cu
import realtime_gp as rt
import numpy as np
import datetime 
import h5py
#import pandas as pd
#from pointingtools import compute_parallactic_angle, altaz2ha 
#from planets import getlocation, getpointing
import warnings
#from astropy.coordinates import AltAz, Angle, EarthLocation, ICRS, SkyCoord, frame_transform_graph
#from astropy import units as u
#import ephem
import matplotlib.pyplot as plt
import time
#import planets
from datetime import timedelta
#import numpy.ma as ma


import Tkinter,tkFileDialog

from Tkinter import *
import ttk

import pickle
import glob

import cPickle

from scipy.optimize import curve_fit

%pylab

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
def read_some_data(datadict,datatype='demod', filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        if datatype=='demod':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='E://software_git_repos/Polaris/polaris_data/demod_data/',parent=root,title='Choose a set of files'))
        if datatype=='raw':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='E://software_git_repos/Polaris/polaris_data/data/',parent=root,title='Choose a set of files'))            
        root.destroy()
    filelist.sort()
    
    if datatype=='demod':
        dlist=[]
        for f in filelist:
            hf=h5py.File(f)
            dlist.append(hf['demod_data'])
        d=concatenate(dlist)
        hf.close() 
        
    if datatype=='raw':
        d=rt.demod.read_raw(filelist)
    
    return d, filelist

In [3]:
def get_file_times(fld):
    startfile = fld[0][:65]+fld[0][71:-2]+'dat'
    endfile = fld[-1][:65]+fld[-1][71:-2]+'dat'
    
    #starttime = os.path.getctime(startfile)
    starttime= os.stat(startfile).st_mtime
    starttime = datetime.datetime.fromtimestamp(starttime)

    #endtime = os.path.getctime(endfile)
    endtime= os.stat(endfile).st_mtime
    endtime = datetime.datetime.fromtimestamp(endtime)
    
    return starttime, endtime

In [4]:
def plot_toi_dict(toi_dict,chanlist=['H1HiAC'],componentlist=['T'],datalist=None, kperv=[1]):
    #function to give a choice of preloaded datasets from dictionary data_dict
    figure()
    if datalist==None:
        datalist=toi_dict.keys()
    if len(componentlist)==1:  # if componentlist is a single value, assume same thing for all chans
        c=componentlist[0]
        componentlist=[c for n in chanlist]
    #set up the color sequence for the plot
    nlines=len(datalist)*len(chanlist)
    colormap = cm.gist_ncar
    gca().set_color_cycle([colormap(i) for i in np.linspace(0, 0.9, nlines)])
    titletxt=[]
    count = 0
    for dname in datalist:
        samprate=1000./(toi_dict[dname]['rev'][1]-toi_dict[dname]['rev'][0])
        titletxt.append(dname)
        for channame,component in zip(chanlist,componentlist):
            try:
                toi=toi_dict[dname][rt.nametochan(channame)][component] #if it's demod data so a dict
                print dname + channame + ' mean value: ', toi.mean()
                t=toi_dict[dname]['azi'][0]+arange(len(toi))/samprate
                if len(kperv)==1 and kperv[0]==1.0:
                    plot(t,toi*kperv[count],label=dname+ '  '+channame+' '+component,lw=2)
                else:
                    plot(t,toi*kperv[count],label=dname+ '  '+channame+' '+component+' gain= '+str(kperv[count]),lw=2)
                if (count+1) < len(kperv):
                    count += 1
            except:
                toi=toi_dict[dname][rt.nametochan(channame)].flatten()#if it's raw data so not a dict
                print dname + ' ' + channame + ' mean value: ', toi.mean()
                t=toi_dict[dname]['azi'][0]+arange(len(toi))/(256.*samprate)
                if len(kperv)==1 and kperv[0]==1.0:
                    plot(t,toi*kperv[count],label=dname+ '  '+channame+' '+component,lw=2)
                else:
                    plot(t,toi*kperv[count],label=dname+ '  '+channame+' '+component+' gain= '+str(kperv[count]),lw=2) 
                if (count+1) < len(kperv):
                    count += 1
                
    title(titletxt)
    legend()
    xlabel('Time [S]')
    if len(kperv) == 1 and kperv[0] == 1:
        ylabel('Output [v]')
    else:
        ylabel('Output [T]')

In [5]:
def plotrawnow(ddict, chan, fld, rstep=50,supply_index=False):
    """
    function to automatically read last science file plot raw data vs encoder
    yrmoday should be a string '20130502' fpath should point to the 
    directory where acq_tel and converter.py were run
    rstep determines how many revolutions to skip between plotted revolutions
    """
    #fld=glob.glob(fpath+'data/'+yrmoday+'/*.dat')
    #fld.sort()
    print fld[-1]
    stats=os.stat(fld[-1])
    if stats.st_size == 10752000:
        dr=ddict#demod.read_raw([fld[-1]],supply_index=supply_index)
        for i in range(0,np.int(np.shape(dr[chan]))[0],rstep):
            plt.plot(dr[chan][i,:],label='rev '+str(i))
        plt.xlabel('encoder position')
        plt.ylabel('Signal V')
        plt.title(chan+' Raw data, every '+str(rstep) + ' revs, file: '+fld[-1])
        plt.legend()
        plt.grid()
        plt.show()
    return dr

In [6]:
ddict = {}

In [7]:
dd, fld = read_some_data(ddict, datatype='raw')

1000
16


In [14]:
name = 'H1HiAC'
chan = rt.nametochan(name)
chan

'ch0'

In [10]:
print np.int(np.shape(dd[chan])[0])

998


In [19]:
stats=os.stat(fld[-1])
rstep = 50
i=0
gain = -4.11
if stats.st_size == 10752000:
    dr=dd#demod.read_raw([fld[-1]],supply_index=supply_index)
    #for i in range(0,np.int(np.shape(dr[chan])[0]),rstep):
    plt.plot(dr[chan][i,:]*gain,label='rev '+str(i))
    plt.xlabel('encoder position')
    plt.ylabel('Signal K')
    #plt.title(name+' Raw data, every '+str(rstep) + ' revs, file: '+fld[-1])
    #plt.title(name+' Raw data, every '+str(rstep) + ' revs, file: '+fld[-1])
    #plt.legend()
    plt.grid()
    plt.show()
else:
    print 'bad file'

In [16]:
data = dd[chan].flatten()

In [17]:
plt.figure()
plt.plot(data)